In [1]:
import glob
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
import planetsca as ps

/Users/elischwat/miniforge3/envs/planetenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/elischwat/miniforge3/envs/planetenv/lib/python3.9/site-packages/pyproj/__init__.py:95: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [14]:
# retrieve the pre-trained ONNX planetsca model from Hugging Face
model = ps.download.retrieve_model_onnx()


from huggingface_hub import hf_hub_download
import joblib
# download model file from Hugging Face
filepath = hf_hub_download(
    repo_id="geo-smart/planetsca_models",
    # filename="random_forest_20220513_binary_174K.joblib"
    # filename="random_forest_20220513_binary_174K_fv.joblib"
    filename="random_forest_20240116_binary_174K.joblib"
)


# open the model that was just downloaded
model = joblib.load(filepath)

In [3]:
ps.download.retrieve_model?

Signature: ps.download.retrieve_model(out_dirpath: Optional[str] = None) -> sklearn.ensemble._forest.RandomForestClassifier
Docstring:
Downloads pre-trained PlanetSCA model from Hugging Face

Parameters:
    out_dirpath: Optional[str]
        Path to directory to save pre-trained model file

Returns
----------
    model: RandomForestClassifier
        The trained PlanetSCA model
File:      ~/miniforge3/envs/planetenv/lib/python3.9/site-packages/planetsca/download.py
Type:      function

In [ ]:
ps_image_filepaths = [
    "/Users/elischwat/Downloads/gothic-co-may13-may16-2023_psscene_analytic_sr_udm2/PSScene/20230515_165304_51_2460_3B_AnalyticMS_SR_clip_file_format.tif"
]

In [ ]:
# where we want to save the resulting SCA geotif images created from the Planet images
output_dirpath = "./SCA/"

In [ ]:
# run the model to predict SCA
sca_image_paths = ps.predict.predict_sca_onnx(
    planet_path=ps_image_filepaths,
    model=model,
    output_dirpath=output_dirpath,
)

In [ ]:
ps_image_filepaths.sort()
sca_image_paths.sort()

for ps_image_filepath, sca_image_filepath in zip(ps_image_filepaths, sca_image_paths):
    ps_image = xr.open_dataset(ps_image_filepath)
    sca_image = xr.open_dataset(sca_image_filepath)

    fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), tight_layout=True)
    ps_image.isel(band=0).band_data.plot(ax=ax1, cmap="Greys_r", add_colorbar=False)
    sca_image.isel(band=0).band_data.plot(ax=ax2, cmap="Greys_r", add_colorbar=False)

    ax1.set_title(ps_image_filepath.split("/")[-1])
    ax2.set_title(sca_image_filepath.split("/")[-1])
